In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn import metrics

In [2]:
#Reading dataset
df=pd.read_csv('DFT.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Cate_name          68 non-null     object 
 1   Cate_HOMO          68 non-null     float64
 2   Cate_LUMO          68 non-null     float64
 3   Cate_MC_O1         68 non-null     float64
 4   Cate_MC_H1         68 non-null     float64
 5   Cate_MC_O2         68 non-null     float64
 6   Cate_MC_H2         68 non-null     float64
 7   Cate_BL_ArO1       68 non-null     float64
 8   Cate_BL_O1H1       68 non-null     float64
 9   Cate_BL_ArO2       68 non-null     float64
 10  Cate_BL_O2H2       68 non-null     float64
 11  Cate_total_dipole  68 non-null     float64
 12  P_name             68 non-null     object 
 13  P_HOMO             68 non-null     float64
 14  P_LUMO             68 non-null     float64
 15  P_MC_P             68 non-null     float64
 16  P_MC_=O            68 non-nu

In [3]:
#Building descriptors
X = df.drop(columns=['Yield', 'Cate_name', 'P_name' ])
print('---Descriptors---')
print(X.head())

y = pd.DataFrame(df['Yield'],columns=['Yield'])
print('---Objective---')
print(y.head())

---Descriptors---
   Cate_HOMO  Cate_LUMO  Cate_MC_O1  Cate_MC_H1  Cate_MC_O2  Cate_MC_H2  \
0  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
1  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
2  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
3  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
4  -5.807916  -0.157824   -0.376813    0.263887   -0.418652    0.276114   

   Cate_BL_ArO1  Cate_BL_O1H1  Cate_BL_ArO2  Cate_BL_O2H2  Cate_total_dipole  \
0       1.36318       0.96622       1.37543       0.96253             2.8219   
1       1.36318       0.96622       1.37543       0.96253             2.8219   
2       1.36318       0.96622       1.37543       0.96253             2.8219   
3       1.36318       0.96622       1.37543       0.96253             2.8219   
4       1.36386       0.96625       1.37631       0.96239             2.5961   

     P_HOMO    P_LUMO    P_MC_P   P_MC_=O    P_MC_

In [4]:
#storage 
data_r2_train =[]
data_RMSE_train = []
data_MAE_train = []
data_r2_test = []
data_RMSE_test = []
data_MAE_test = []
best_model_parametors = []
data_cv_score = []

#SVM_a_X,log(y)
for i in range(10):
    seed=i
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    #autoscaling
    a_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    
    log_y_train = np.log(y_train)
    
    param_svm={'C':[0.1,1,5,10,50,100,200,300,400,500,750,1000],
               'gamma':[100,10,1,0.1,0.01,0.001,0.0001,0.00001]}
    reg_svm = GridSearchCV(SVR(kernel='rbf', epsilon=0.3), param_grid=param_svm, cv=5, n_jobs=16)
    reg_svm.fit(a_X_train,log_y_train['Yield'])
    reg_best = reg_svm.best_estimator_
    y_pred1 = np.exp(reg_best.predict(a_X_train))
    y_pred2 = np.exp(reg_best.predict(a_X_test))
    
    #train
    r2_train = metrics.r2_score(y_train, y_pred1)
    RMSE_train = metrics.root_mean_squared_error(y_train, y_pred1)
    MAE_train =  metrics.mean_absolute_error(y_train, y_pred1)
    #test
    r2_test = metrics.r2_score(y_test, y_pred2)
    RMSE_test = metrics.root_mean_squared_error(y_test, y_pred2)
    MAE_test = metrics.mean_absolute_error(y_test, y_pred2)
    
    parametors = reg_svm.best_params_
    best_model_parametors.append(parametors)
    cv_score = reg_svm.best_score_
    data_cv_score.append(cv_score)
    
    data_r2_train.append(r2_train)
    data_RMSE_train.append(RMSE_train)
    data_MAE_train.append(MAE_train)
    data_r2_test.append(r2_test)
    data_RMSE_test.append(RMSE_test)
    data_MAE_test.append(MAE_test)
    
    print('----------------------')
    print('seed:', seed)
    print("Best Model Parameter:",reg_svm.best_params_)
    print("Best Model Score:",reg_svm.best_score_)
    print('R2_test:', r2_test)
print('R2_train_means:', sum(data_r2_train)/10)
print('CV_score_means:', sum(data_cv_score)/10)
print('R2_test_means:', sum(data_r2_test)/10)

----------------------
seed: 0
Best Model Parameter: {'C': 750, 'gamma': 0.001}
Best Model Score: 0.9233910346193381
R2_test: 0.7865703573994896
----------------------
seed: 1
Best Model Parameter: {'C': 500, 'gamma': 0.001}
Best Model Score: 0.904396703102665
R2_test: 0.8948810967899752
----------------------
seed: 2
Best Model Parameter: {'C': 750, 'gamma': 0.001}
Best Model Score: 0.9225171398751293
R2_test: 0.839505298264101
----------------------
seed: 3
Best Model Parameter: {'C': 400, 'gamma': 0.001}
Best Model Score: 0.9036674903081867
R2_test: 0.8183237326924493
----------------------
seed: 4
Best Model Parameter: {'C': 300, 'gamma': 0.001}
Best Model Score: 0.8678396502280726
R2_test: 0.7058886959771067
----------------------
seed: 5
Best Model Parameter: {'C': 750, 'gamma': 0.001}
Best Model Score: 0.8757930624970957
R2_test: 0.8026089766681866
----------------------
seed: 6
Best Model Parameter: {'C': 300, 'gamma': 0.001}
Best Model Score: 0.8795715351702673
R2_test: 0.6945

In [5]:
data_r2_train_pd = pd.DataFrame(data= data_r2_train, columns=['r2_train'])
data_RMSE_train_pd = pd.DataFrame(data=data_RMSE_train, columns=['RMSE_train'])
data_MAE_train_pd = pd.DataFrame(data=data_MAE_train, columns=['MAE_train'])
data_r2_test_pd = pd.DataFrame(data=data_r2_test, columns=['r2_test'])
data_RMSE_test_pd = pd.DataFrame(data=data_RMSE_test, columns=['RMSE_test'])
data_MAE_test_pd = pd.DataFrame(data=data_MAE_test, columns=['MAE_test'])
data_cv_score_pd = pd.DataFrame(data=data_cv_score, columns=['cv_score'])
data_parametors = pd.DataFrame([best_model_parametors])

data_all = pd.concat([data_r2_train_pd, data_RMSE_train_pd, data_MAE_train_pd, data_r2_test_pd, 
                      data_RMSE_test_pd, data_MAE_test_pd, data_cv_score_pd],
                     axis=1, join='inner')

data_all.loc['mean'] = data_all.mean()
data_all.loc['std'] = data_all.std(ddof=0)
print(data_all)

data_all.to_csv('../../score/DFT//SVM_score.csv')

      r2_train  RMSE_train  MAE_train   r2_test  RMSE_test   MAE_test  \
0     0.890075    9.002452   5.946908  0.786570  16.707942  10.003221   
1     0.847029   11.761509   7.212636  0.894881  10.091791   6.074660   
2     0.875832   10.717635   6.636266  0.839505  12.458258   8.108422   
3     0.855727   11.600220   7.263139  0.818324  13.204661   8.892391   
4     0.859049   11.023330   6.547547  0.705889  17.684334  10.552375   
5     0.851115   10.979888   7.005199  0.802609  15.494120  10.121779   
6     0.826190   11.943252   7.688672  0.694530  19.173445  10.518529   
7     0.834345   12.587636   7.476077  0.750875  15.027758   9.476802   
8     0.852251   11.403334   6.371336  0.720879  16.608353  11.187935   
9     0.787726   14.667242   9.450777  0.924235   7.306754   4.919206   
mean  0.847934   11.568650   7.159856  0.793830  14.375742   8.985532   
std   0.025317    1.307290   0.874248  0.070578   3.295111   1.858884   

      cv_score  
0     0.923391  
1     0.904397  